In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import sys
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('/content/drive/Shareddrives/CS2470/books_datasetv2.csv')
df.head

<bound method NDFrame.head of       Unnamed: 0           ISBN  \
0              0  9781419726552   
1              1  9780982301821   
2              2  9780316407076   
3              3  9780718088804   
4              4  9780451495426   
...          ...            ...   
1641        1666  9780451491640   
1642        1667  9780008263737   
1643        1668  9780525522478   
1644        1669  9780062676832   
1645        1670  9780062570604   

                                                  title  series  \
0     F*ck, That's Delicious: An Annotated Guide to ...       0   
1     10-Day Green Smoothie Cleanse: Lose Up to 15 P...       0   
2                                           15th Affair       1   
3       42 Faith: The Rest of the Jackie Robinson Story       0   
4     7 Lessons from Heaven: How Dying Taught Me to ...       0   
...                                                 ...     ...   
1641                               Archangel's Prophecy       1   
1642         

In [ ]:
import re
# clean title data

def clean_data(df, col, clean_col):

    # change to lower and remove spaces on either side
    df[clean_col] = df[col].apply(lambda x: x.lower().strip())

    # remove extra spaces in between
    df[clean_col] = df[clean_col].apply(lambda x: re.sub(' +', ' ', x))

    # remove punctuation
    df[clean_col] = df[clean_col].apply(lambda x: re.sub('[^a-zA-Z]', ' ', x))

   
    return df

# calling function
df = clean_data(df, 'title', 'title_clean')
df.head()

,Unnamed: 0,ISBN,title,series,author,rating,genre,publish_company,format,number_of_pages,publish_month,publish_year,NYT_bestseller,top5,length_of_title,top_authors,title_clean
0,0,9781419726552,"F*ck, That's Delicious: An Annotated Guide to ...",0,Action Bronson,4.17,food-and-drink,Harry N. Abrams,Hardcover,192,September,2017,1.0,0,57,0,f ck that s delicious an annotated guide to ...
1,1,9780982301821,10-Day Green Smoothie Cleanse: Lose Up to 15 P...,0,J.J. Smith,4.11,health,Adiva Publishing,Paperback,167,February,2014,1.0,0,63,0,day green smoothie cleanse lose up to p...
2,2,9780316407076,15th Affair,1,James Patterson,4.09,mystery,Hachette,Hardcover,351,May,2016,1.0,1,11,1,th affair
3,3,9780718088804,42 Faith: The Rest of the Jackie Robinson Story,0,Ed Henry,4.16,biography,Harper Collins,Hardcover,368,April,2017,1.0,1,47,0,faith the rest of the jackie robinson story
4,4,9780451495426,7 Lessons from Heaven: How Dying Taught Me to ...,0,Mary C. Neal,4.08,non-fiction,Convergent Books,Paperback,288,September,2017,1.0,0,68,0,lessons from heaven how dying taught me to ...


In [ ]:
# tokenization
import nltk
nltk.download('punkt')
df["title_token"] = df.loc[:, "title_clean"].apply(nltk.word_tokenize)

# list to strings
#df["title_token"] = [','.join(map(str, l)) for l in df['title_token']]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
df["title_token"]

0       [f, ck, that, s, delicious, an, annotated, gui...
1       [day, green, smoothie, cleanse, lose, up, to, ...
2                                            [th, affair]
3       [faith, the, rest, of, the, jackie, robinson, ...
4       [lessons, from, heaven, how, dying, taught, me...
                              ...                        
1641                             [archangel, s, prophecy]
1642              [sunshine, at, the, comfort, food, caf]
1643                                  [depth, of, winter]
1644                           [the, sentence, is, death]
1645                                      [dread, nation]
Name: title_token, Length: 1646, dtype: object

In [ ]:
title = df["title_token"]
label = df["NYT_bestseller"]
train_x, test_x, train_y, test_y = train_test_split(title, label, test_size=0.3)
train_title = [i for i in train_x]
train_label = [i for i in train_y]
test_title = [i for i in test_x]
test_label = [i for i in test_y]

In [ ]:
def build_vocab(sentences):
	tokens = []
	for s in sentences: tokens.extend(s) 
	all_words = sorted(list(set(tokens)))  
	vocab =  {word:i for i,word in enumerate(all_words)}
	return vocab

def convert_to_id(vocab, sentences):
  id =[]
  for j in range(len(sentences)) :
    id.append([vocab[i] for i in sentences[j]])
  return id

In [ ]:
vocab_all = build_vocab(title) # dictionary
id_all =  convert_to_id(vocab_all, title)

In [ ]:
## vocab, id, id with padding

train_vocab = build_vocab(train_title)

train_id = convert_to_id(train_vocab, train_title)
train_id_padded = tf.keras.preprocessing.sequence.pad_sequences(train_id, padding="post")
train_id_padded = tf.cast(train_id_padded, dtype = tf.float64)

train_label = tf.cast(train_label, dtype = tf.float64)

print(train_vocab)
print(train_id_padded)

test_vocab = build_vocab(test_title)
test_id = convert_to_id(test_vocab, test_title)
test_id_padded = tf.keras.preprocessing.sequence.pad_sequences(test_id, padding="post")
test_id_padded = tf.cast(test_id_padded, dtype = tf.float64)
test_label = tf.cast(test_label, dtype = tf.float64)


{'a': 0, 'about': 1, 'abraham': 2, 'absolute': 3, 'abundance': 4, 'abyss': 5, 'academy': 6, 'accelerations': 7, 'achieve': 8, 'acker': 9, 'adam': 10, 'addicted': 11, 'addiction': 12, 'administration': 13, 'adult': 14, 'adultolescence': 15, 'adventure': 16, 'adventures': 17, 'adversity': 18, 'afar': 19, 'affair': 20, 'affairs': 21, 'affluence': 22, 'affordable': 23, 'afghanistan': 24, 'african': 25, 'after': 26, 'afterlives': 27, 'again': 28, 'against': 29, 'age': 30, 'agenda': 31, 'agent': 32, 'ageproof': 33, 'akata': 34, 'al': 35, 'alamo': 36, 'alaska': 37, 'albert': 38, 'alchemist': 39, 'alert': 40, 'alexander': 41, 'alice': 42, 'alive': 43, 'all': 44, 'allowed': 45, 'almost': 46, 'alone': 47, 'always': 48, 'am': 49, 'amazing': 50, 'america': 51, 'american': 52, 'amnesia': 53, 'among': 54, 'an': 55, 'anatomy': 56, 'ancient': 57, 'and': 58, 'andy': 59, 'angels': 60, 'anger': 61, 'angler': 62, 'animal': 63, 'animals': 64, 'animators': 65, 'anna': 66, 'annex': 67, 'anomaly': 68, 'anonym

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPool1D, GRU, Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from collections import Counter
from sklearn.model_selection import train_test_split


In [ ]:
tf.random.set_seed(1001)
batch_size = 100 
embedding_size = 128		
hidden_size = 64
vocab_size = len(train_id_padded)

# expand model / incorporte other types of variables / compare diff. strategies 

model = Sequential()
embedding_layer = Embedding(input_dim = 5000, mask_zero=True, output_dim = hidden_size)
model.add(embedding_layer)
model.add(Conv1D(filters=256, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPool1D(pool_size=2, strides=1, padding='same'))
model.add(Conv1D(filters=128, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPool1D(pool_size=4))
model.add(Conv1D(filters=64, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPool1D(pool_size=4))
model.add(GRU(hidden_size))
model.add(Dropout(0.4))
model.add(Flatten())
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['acc'])
model.fit(train_id_padded, train_label, epochs=5, batch_size=batch_size)
model.summary()



Epoch 1/5
12/12 [==============================] - 5s 118ms/step - loss: 0.6575 - acc: 0.6632
Epoch 2/5
12/12 [==============================] - 1s 114ms/step - loss: 0.6377 - acc: 0.6693
Epoch 3/5
12/12 [==============================] - 1s 114ms/step - loss: 0.6245 - acc: 0.6693
Epoch 4/5
12/12 [==============================] - 1s 113ms/step - loss: 0.6041 - acc: 0.6762
Epoch 5/5
12/12 [==============================] - 1s 114ms/step - loss: 0.5018 - acc: 0.7179
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          320000    
                                                                 
 conv1d (Conv1D)             (None, None, 256)         65792     
                                                                 
 max_pooling1d (MaxPooling1D  (None, None, 256)        0         
 )                                              

In [ ]:
model(train_id_padded)

<tf.Tensor: shape=(1152, 1), dtype=float32, numpy=
array([[0.07647231],
       [0.24304086],
       [0.1360879 ],
       ...,
       [0.46613434],
       [0.4720856 ],
       [0.018374  ]], dtype=float32)>

In [ ]:
# y_score = model.predict(test_id_padded)
# y_score = [[1 if i == max(sc) else 0 for i in sc] for sc in y_score]

# n_right = 0
# for i in range(len(y_score)):
#     if all(y_score[i][j] == test_label[i] for j in range(len(y_score[i]))):
#         n_right += 1

# print("Accuracy in test: %.2f%%" % ((n_right/float(len(test_y)) * 100)))

In [ ]:
y_score = model.predict(test_id_padded)
y_score = [[1 if i >= 0.5 else 0 for i in sc] for sc in y_score] # threshold 0.5 okay?

n_right = 0
for i in range(len(y_score)):
    if all(y_score[i][j] == test_label[i] for j in range(len(y_score[i]))):
        n_right += 1

print("Accuracy in test: %.2f%%" % ((n_right/float(len(test_y)) * 100)))

Accuracy in test: 65.79%
